In [1]:
import os
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix

In [11]:
res_base_dir = '/Users/hragnarsd/Documents/masters/results'
folds = [0, 1, 2, 3, 4]
result_folders = [f'{res_base_dir}/fold{fold}' for fold in folds]

In [12]:
max_epochs = [10, 14, 120, 250, 152]
model_names = [f'res_simple_adamw_lt_2d.k{fold}.lr_0.001.batch_128_aug_bal_hist_drop0.5_e{e}' for fold, e in zip(folds, max_epochs)]

In [28]:
def print_all_metrics(true, pred):
    print(classification_report(true, pred))
    print(confusion_matrix(true, pred))
    print()
    print(f"f1 macro: {f1_score(true, pred, average='macro'):.2f}")
    print(f"f1 micro: {f1_score(true, pred, average='micro'):.2f}")
    print(f"f1 binary: {f1_score(true, pred, average='binary'):.2f}")
    print(f"f1 weighted: {f1_score(true, pred, average='weighted'):.2f}")
    print()
    print(f"roc_auc macro: {roc_auc_score(true, pred, average='macro'):.2f}")
    print(f"roc_auc micro: {roc_auc_score(true,  pred, average='micro'):.2f}")
    print(f"roc_auc weighted: {roc_auc_score(true, pred, average='weighted'):.2f}")
    print()
    print(f"balanced accuracy: {balanced_accuracy_score(true, pred):.2f}")

In [89]:
def res_for_patient(samples, targets, preds, patient_id=30):
    patient_targets = []
    patient_preds = []
    patient_samples = []
    for s, t, p in zip(samples, targets, preds):
        # print(s)
        curr_patient_id = s.split('_')[0] 
        if int(curr_patient_id) == patient_id:
            patient_targets.append(t)
            patient_preds.append(p)
            patient_samples.append(s)
    return patient_samples, patient_targets, patient_preds, 

In [152]:
def get_results(base_model, res_base_dir):
    """
    base_model is the name of the model for any one fold, e.g. the first fold, without epoch info
    """
    model_name_p1 = base_model.split('.k')[0]
    model_name_p2 = base_model.split('.lr')[1]
    print(model_name_p1, model_name_p2)

    folds = [0, 1, 2, 3, 4]
    result_folders = [f'{res_base_dir}/fold{fold}' for fold in folds]
    base_model_names = [f'{model_name_p1}.k{fold}.lr{model_name_p2}' for fold in folds]

    
    # Get max epochs
    max_epochs = []
    for fold_dir, model_name in zip(result_folders, base_model_names):
        max_epoch = -1
        for file in os.listdir(fold_dir):
            file_wout_epoch, epoch = file.split('_e')
            if file_wout_epoch == model_name:
                if epoch > max_epoch:
                    max_epoch = epoch
        max_epochs.append(max_epoch)
    
    print('max epochs', max_epochs)
    # Get results, given max epochs
    total_val_preds = []
    total_val_targets = []
    total_val_samples = []
    model_names = [f'{base_name}_e{epoch}' for base_name, epoch in zip(base_model_names, max_epochs)]
    for result_folder, model_name in zip(result_folders, model_names):
        res_subdir = os.path.join(result_folder, model_name)
        val_preds = np.load(os.path.join(res_subdir, 'val_preds_' + model_name + '.npy'))
        total_val_preds.extend(val_preds)
        val_targets = np.load(os.path.join(res_subdir, 'val_targets_' + model_name + '.npy'))
        total_val_targets.extend(val_targets)
        val_samples = np.load(os.path.join(res_subdir, 'val_samples_' + model_name + '.npy'))
        total_val_samples.extend(val_samples)
    
    print_all_metrics(total_val_targets, total_val_preds)
    return total_val_samples, total_val_targets, total_val_preds

In [46]:
# Augment, histo, balanced data per epoch

In [72]:
unique_val_samples = [int(s) for s in np.unique(patients)]
print(sorted(unique_val_samples))
dict = 

[30, 31, 34, 36, 37, 38, 39, 41, 48, 49, 50, 55, 56, 57, 58, 60, 61, 62, 67, 68, 69, 71, 72, 73, 78, 79, 82, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 103, 104, 105, 107, 108, 110, 111, 112, 113, 115, 118, 119, 120, 121, 123, 124, 130, 131, 132, 133, 135, 136, 137, 138, 139, 143, 144, 145, 146, 147, 149, 150, 151, 155, 157, 159, 160, 161, 162, 164, 165, 167, 168, 170, 172, 174, 175, 176, 177, 178, 179, 183, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 198, 200, 202, 203, 204, 208, 209, 210, 211, 213, 214, 215, 216, 219, 221, 223, 225, 227, 228, 230, 231, 232, 233, 234, 235, 237, 239]


In [134]:
patient = 192
frames = []
for s in samples:
    if s.startswith(str(patient)):
        frames.append(int(s.split('_')[1]))
print(frames)

[17, 18, 24, 25, 27, 35, 45, 46, 47, 48, 57, 66, 67]


In [124]:
for s in samples:
    if s.startswith('159'):
        print(s)

159_46
159_47
159_48
159_49
159_50
159_56
159_57
159_58
159_59
159_70
159_90
159_91
159_124


In [142]:
preds_per_sample = []
targets_per_sample = []
for id in unique_val_samples:
# for id in [135, 137, 155, 159, 160, 162, 214]:
    p_samples, p_targets, p_preds = res_for_patient(samples, targets, preds, patient_id=id)
    p_preds = np.asarray(p_preds)

    percentage_label_1 = p_preds.sum()/len(p_preds)
    if p_targets[0] == 1 and percentage_label_1 < 0.4:
        print(id, percentage_label_1, p_targets[0])
    if percentage_label_1 >= 0.5:
        pred = 1
    else:
        pred = 0

    preds_per_sample.append(pred)
    targets_per_sample.append(p_targets[0]) # always same target val

print_all_metrics(targets_per_sample, preds_per_sample)

104 0.15384615384615385 1
124 0.07692307692307693 1
137 0.0 1
159 0.0 1
160 0.0 1
192 0.3076923076923077 1
195 0.21428571428571427 1
214 0.0 1
94 0.3333333333333333 1
              precision    recall  f1-score   support

           0       0.89      0.82      0.85        87
           1       0.68      0.79      0.73        43

    accuracy                           0.81       130
   macro avg       0.78      0.80      0.79       130
weighted avg       0.82      0.81      0.81       130

[[71 16]
 [ 9 34]]

f1 macro: 0.79
f1 micro: 0.81
f1 binary: 0.73
f1 weighted: 0.81

roc_auc macro: 0.80
roc_auc micro: 0.80
roc_auc weighted: 0.80

balanced accuracy: 0.80


In [80]:
# samples always same for all models (could actually just store 1x => order same bc don't shuffle valid data)
samples, targets, preds = get_results('res_simple_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_bal_hist_drop0.5
max epochs [10, 14, 120, 250, 152]
              precision    recall  f1-score   support

           0       0.87      0.78      0.82      1694
           1       0.64      0.77      0.70       846

    accuracy                           0.78      2540
   macro avg       0.75      0.78      0.76      2540
weighted avg       0.79      0.78      0.78      2540

[[1326  368]
 [ 196  650]]

f1 macro: 0.76
f1 micro: 0.78
f1 binary: 0.70
f1 weighted: 0.78

roc_auc macro: 0.78
roc_auc micro: 0.78
roc_auc weighted: 0.78

balanced accuracy: 0.78


In [65]:
patients = [s.split('_')[0] for s in samples]
# print(patients[0])
x = []
x.append(patients[0])
for i in range(len(patients)):
    if i > 0:
        if patients[i] != patients[i - 1]:
            # print(patients[i])
            x.append(patients[i])
print('len x', len(x))
print('len unique', len(np.unique(patients)))
print('no repeated patients', len(x) - len(np.unique(patients))) # 65 repeated patients (!)

len x 195
len unique 130
no repeated patients 65


In [64]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.001.batch_128_aug_w_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_w_hist_drop0.5
max epochs [38, 152, 226, 38, 38]
              precision    recall  f1-score   support

           0       0.83      0.81      0.82      1694
           1       0.63      0.67      0.65       846

    accuracy                           0.76      2540
   macro avg       0.73      0.74      0.74      2540
weighted avg       0.77      0.76      0.76      2540

[[1367  327]
 [ 278  568]]

f1 macro: 0.74
f1 micro: 0.76
f1 binary: 0.65
f1 weighted: 0.76

roc_auc macro: 0.74
roc_auc micro: 0.74
roc_auc weighted: 0.74

balanced accuracy: 0.74


In [33]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.001.batch_128_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_hist_drop0.5
max epochs [162, 164, 64, 276, 66]
              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1694
           1       0.65      0.70      0.67       846

    accuracy                           0.78      2540
   macro avg       0.75      0.76      0.75      2540
weighted avg       0.78      0.78      0.78      2540

[[1381  313]
 [ 257  589]]

f1 macro: 0.75
f1 micro: 0.78
f1 binary: 0.67
f1 weighted: 0.78

roc_auc macro: 0.76
roc_auc micro: 0.76
roc_auc weighted: 0.76

balanced accuracy: 0.76


In [36]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.0001.batch_128_aug_bal_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.0001.batch_128_aug_bal_hist_drop0.5
max epochs [18, 42, 244, 202, 266]
              precision    recall  f1-score   support

           0       0.83      0.76      0.79      1694
           1       0.58      0.68      0.63       846

    accuracy                           0.73      2540
   macro avg       0.70      0.72      0.71      2540
weighted avg       0.75      0.73      0.74      2540

[[1282  412]
 [ 269  577]]

f1 macro: 0.71
f1 micro: 0.73
f1 binary: 0.63
f1 weighted: 0.74

roc_auc macro: 0.72
roc_auc micro: 0.72
roc_auc weighted: 0.72

balanced accuracy: 0.72


In [38]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.0001.batch_128_aug_w_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.0001.batch_128_aug_w_hist_drop0.5
max epochs [40, 54, 112, 198, 254]
              precision    recall  f1-score   support

           0       0.86      0.70      0.77      1694
           1       0.56      0.76      0.65       846

    accuracy                           0.72      2540
   macro avg       0.71      0.73      0.71      2540
weighted avg       0.76      0.72      0.73      2540

[[1189  505]
 [ 201  645]]

f1 macro: 0.71
f1 micro: 0.72
f1 binary: 0.65
f1 weighted: 0.73

roc_auc macro: 0.73
roc_auc micro: 0.73
roc_auc weighted: 0.73

balanced accuracy: 0.73


In [37]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.001.batch_1280.0510.0_aug_bal_hist_drop0.5', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_1280.0510.0_aug_bal_hist_drop0.5
max epochs [12, 6, 30, 12, 16]
              precision    recall  f1-score   support

           0       0.83      0.70      0.76      1694
           1       0.54      0.71      0.61       846

    accuracy                           0.70      2540
   macro avg       0.68      0.70      0.68      2540
weighted avg       0.73      0.70      0.71      2540

[[1179  515]
 [ 244  602]]

f1 macro: 0.68
f1 micro: 0.70
f1 binary: 0.61
f1 weighted: 0.71

roc_auc macro: 0.70
roc_auc micro: 0.70
roc_auc weighted: 0.70

balanced accuracy: 0.70


In [39]:
get_results('res_simple_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_drop0.25', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_bal_hist_drop0.25
max epochs [84, 128, 110, 130, 176]
              precision    recall  f1-score   support

           0       0.85      0.82      0.83      1694
           1       0.66      0.71      0.68       846

    accuracy                           0.78      2540
   macro avg       0.75      0.76      0.76      2540
weighted avg       0.79      0.78      0.78      2540

[[1384  310]
 [ 245  601]]

f1 macro: 0.76
f1 micro: 0.78
f1 binary: 0.68
f1 weighted: 0.78

roc_auc macro: 0.76
roc_auc micro: 0.76
roc_auc weighted: 0.76

balanced accuracy: 0.76


In [40]:
get_results('res_simple_adamw_lt_2d.k4.lr_0.001.batch_128_aug_w_hist_drop0.25', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_w_hist_drop0.25
max epochs [170, 8, 234, 134, 210]
              precision    recall  f1-score   support

           0       0.84      0.80      0.82      1694
           1       0.63      0.69      0.66       846

    accuracy                           0.76      2540
   macro avg       0.73      0.74      0.74      2540
weighted avg       0.77      0.76      0.76      2540

[[1348  346]
 [ 261  585]]

f1 macro: 0.74
f1 micro: 0.76
f1 binary: 0.66
f1 weighted: 0.76

roc_auc macro: 0.74
roc_auc micro: 0.74
roc_auc weighted: 0.74

balanced accuracy: 0.74


In [41]:
get_results('res_simple_adamw_lt_2d.k4.lr_0.001.batch_128_aug_bal_hist_drop0.6', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_bal_hist_drop0.6
max epochs [16, 230, 110, 250, 144]
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      1694
           1       0.65      0.71      0.68       846

    accuracy                           0.78      2540
   macro avg       0.75      0.76      0.75      2540
weighted avg       0.78      0.78      0.78      2540

[[1373  321]
 [ 246  600]]

f1 macro: 0.75
f1 micro: 0.78
f1 binary: 0.68
f1 weighted: 0.78

roc_auc macro: 0.76
roc_auc micro: 0.76
roc_auc weighted: 0.76

balanced accuracy: 0.76


In [42]:
get_results('res_simple_adamw_lt_2d.k4.lr_0.001.batch_128_aug_w_hist_drop0.6', res_base_dir)

res_simple_adamw_lt_2d _0.001.batch_128_aug_w_hist_drop0.6
max epochs [60, 44, 106, 276, 72]
              precision    recall  f1-score   support

           0       0.85      0.82      0.83      1694
           1       0.66      0.71      0.68       846

    accuracy                           0.78      2540
   macro avg       0.75      0.76      0.76      2540
weighted avg       0.78      0.78      0.78      2540

[[1382  312]
 [ 247  599]]

f1 macro: 0.76
f1 micro: 0.78
f1 binary: 0.68
f1 weighted: 0.78

roc_auc macro: 0.76
roc_auc micro: 0.76
roc_auc weighted: 0.76

balanced accuracy: 0.76


### Other methods

In [153]:
get_results('resnet_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist', res_base_dir)

resnet_adamw_lt_2d _0.001.batch_128_aug_bal_hist
max epochs [-1, -1, -1, -1, -1]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hragnarsd/Documents/masters/results/fold0/resnet_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_e-1/val_preds_resnet_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_e-1.npy'

In [144]:
get_results('resnet_adam_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist', res_base_dir)

resnet_adam_lt_2d _0.001.batch_128_aug_bal_hist
max epochs [-1, -1, -1, -1, -1]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hragnarsd/Documents/masters/results/fold0/resnet_adam_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_e-1/val_preds_resnet_adam_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_e-1.npy'

In [ ]:
get_results('conv_adamw_lt_2d.k0.lr_0.001.batch_128_aug_bal_hist_drop0.5', res_base_dir)